In [1]:
# Create context manager class
class MyContext:
    def __init__(self):
        print('init running...')
        self.obj = None
    
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return object'
        return self.obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occurred: {exc_type}, {exc_value}')
        return True    # if False than the exception will be raised

In [2]:
# create instance of the class
ctx = MyContext()
print('created context...', '\n')

with ctx as obj:   # with MyContext() as obj
    print(obj)
    raise ValueError()

init running...
created context... 

entering context...
the Return object
exiting context...
*** Error occurred: <class 'ValueError'>, 


<br>

In [3]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file...')
        self.file.close()
        return False

In [4]:
with File('test.txt', 'w') as fh:
    fh.write('qwerty')

opening file...
closing file...


In [5]:
with File('test.txt', 'r') as fh:
    print(fh.readlines())

opening file...
['qwerty']
closing file...


<br>
<br>
<br>

In [6]:
import decimal

<br>

How this is usually work

In [7]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [8]:
# let's change precision
decimal.getcontext().prec = 14

# check
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [9]:
# return precision to the initial value
decimal.getcontext().prec = 28

In [10]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 2
print(decimal.Decimal(1) / decimal.Decimal(3))

decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.33
0.3333333333333333333333333333


<br>

example 1: simplest

In [11]:
# let's create context manager class
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_td):
        decimal.getcontext().prec = self.current_prec
        return False

In [12]:
with precision(4):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [13]:
with decimal.localcontext() as ctx:
    ctx.prec = 5
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.33333
0.3333333333333333333333333333


<br>

example 2: time measurement

In [14]:
from time import perf_counter, sleep

In [15]:
class Timer:
    def __init__(self):
        self.elapsed = 0
    
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [16]:
with Timer() as timer:
    sleep(1)
    
print(timer.start)
print(timer.elapsed)
print(timer.stop)

9902.034803222
1.001064754998879
9903.035867977


<br>

example 3: writing to a file

In [17]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
        
    def __exit__(self, exc_type, exc_value, exc_td):
        sys.stdout = self._current_stdout
        self._file.close()
        return False

In [18]:
with OutToFile('test.txt'):
    print('Line 1')
    print('Line 2')

In [19]:
with open('test.txt') as fh:
    for ln in fh:
        print(ln, end='')

Line 1
Line 2


In [20]:
# Let's look what is sys.stdout is
print(type(sys.stdout))
print(sys.stdout)

<class 'ipykernel.iostream.OutStream'>


<br>

example 4: surrounding a text with a html-tag, including a tag inside a tag

In [21]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
        
    def __enter__(self):
        print(f'<{self._tag}>', end='')
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')
        return False

In [22]:
with Tag('p'):
    print('some ', end='')
    with Tag('b'):
        print('bold', end='')
    print(' text', end='')

<p>some <b>bold</b> text</p>

<br>

example 5: more complicated complex manager - creating nested list

```
Title
→ Items 1
   → sub item 1a
   → sub item 1b
→ Items 2
   → sub item 2a
   → sub item 2b
```

In [23]:
class ListMaker:
    def __init__(self, title, prefix='→ ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
    
    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [24]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('sub item 1a')
        lm.print_item('sub item 2a')
    lm.print_item('Item 2')

Items
   → Item 1
      → sub item 1a
      → sub item 2a
   → Item 2
